# `GiRaFFE_NRPy`: Basic equations and modules

## Authors: Patrick Nelson &

`GiRaFFE_NRPy` rewrites the original `GiRaFFE` in a manner made to be more compatible with NRPy+, allowing us to generate large, complicated portions of the code quickly and easily. It solves the equations of General-Relativistic, Force-Free Electrodynamics (GRFFE), an special case of General-Relativistic Magnetohydrodynamics (GRMHD) in which pressures and densities are zero. 

To do so, initial data must first be generated. Conventionally, this is given in the forms of equations describing the vector potential $A_i$ and electric field $E^i$. From these expressions, $A_i$ is set directly; the magnetic field $B^i$ is computed from $A_i$ analytically and, with $E^i$, used to calculate the initial Valencia three-velocity $\bar{v}^i$. Then, the densitized Poynting flux $\tilde{S}_i$ is calculated from $\bar{v}^i$. 

At each timestep, the conservative variables $A_i$ and $\tilde{S}_i$ are advanced through time using the method of lines (MoL). Reconstruction methods, solving the Riemann problem, and finite-differencing methods are used to calculate spatial derivatives, and an RK solver handles the time evolution. Then, we update the primitive variables. $B^i$ is numerically calculated from $A_i$ and fixes are applied to $\tilde{S}_i$ before calculating $\bar{v}^i$ at the new timestep. Another algorithm is applied to $\bar{v}^i$ to preserve current sheets in the $x$-$y$ plane, and $\tilde{S}_i$ is updated to be consistent with these changes. 

Below is a list of modules, ordered to approximately reflect the order in which they would come into play in a typical run.

**List of Modules:**
1. Overview
    1. [You are here!](Tutorial-GiRaFFE_NRPy-Overview.ipynb)
1. Initial Data
    1. Exact Wald
        * This initial data represents a spinning black hole in a uniform magnetic field aligned with the black hole's spin axis. [Tutorial](Tutorial-GiRaFFEfood_NRPy_Exact_Wald.ipynb)
        * This module currently contains a routine to calculate the initial $\tilde{S}_i$ from the initial $\bar{v}^i$, but this is probably redundant with the new P2C solver.
    1. Aligned Rotator
        * This flat-space test represents a toy model of a black hole magnetosphere. [Tutorial](Tutorial-GiRaFFEfood_NRPy_Aligned_Rotator.ipynb)
    1. 1-D test: Alfv&eacute;n Wave
        * This test demonstrates the propagation of a specific MHD wave. [Tutorial](Tutorial-GiRaFFEfood_NRPy_1D_tests.ipynb)
    1. More initial data to come!
1. Evolution
    1. Interpolate metric on cell faces
        * Basic interpolation of the metric gridfunctions at cell centers to cell faces given a specific direction. [Tutorial](Tutorial-GiRaFFE_NRPy-FCVAL.ipynb)
    1. Reconstruct primitives on cell faces
        * This uses the piecewise-parabolic method to reconstruct gridfunctions on cell faces and apply slope-limiting algorithms mitigate the Gibbs phenomenon and preserve shocks. [Tutorial](Tutorial-GiRaFFE_NRPy-PPM.ipynb)
    1. Solve the Riemann Problem for $\tilde{S}_i$: <font color='Green'><b>Validated</b></font>
        * Use the reconstructed velocities and magnetic fields on the left and right sides of the cell faces to solve the Riemann problem and compute the flux of $\tilde{S}_i$ on the interface. [Tutorial](Tutorial-GiRaFFE_NRPy-Stilde-flux.ipynb), [Unit test](Tutorial-Start_to_Finish-GiRaFFE_NRPy-Stilde_flux.ipynb)
    1. Solve the Riemann Problem for $A_i$: <font color='Orange'><b>In Progress</b></font>
        * Here, we apply a similar algorithm as in the previous step to compute the electric field $E_i = \epsilon_{ijk} v^i B^i$. [Tutorial](Tutorial-GiRaFFE_NRPy-Induction_Equation.ipynb)
    1. Calculate the magnetic field: <font color='green'><b>Validated</b></font>
        * Compute $B^i = \epsilon^{ijk} \partial_j A_k$ using standard finite-differencing methods. Care must be taken in the ghost zones to shift the template appropriately to avoid accessing points that are not stored in memory. [Tutorial](Tutorial-GiRaFFE_NRPy-A2B.ipynb), [Unit test](Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.ipynb)
    1. Recalculate $\bar{v}^i$ and apply fixes to $\tilde{S}_i$ and $\bar{v}^i$: <font color='green'><b>Validated</b></font>
        * Apply fixes to $\tilde{S}_i$, then recompute $\bar{v}^i$ at the next time step. Then, apply fixes to $\bar{v}^i$ to preserve current sheets, and recompute $\tilde{S}_i$ to match. [Tutorial](Tutorial-GiRaFFE_NRPy-C2P_P2C.ipynb), [Unit test](Tutorial-Start_to_Finish-GiRaFFE_NRPy-C2P_P2C.ipynb)
    1. Apply boundary conditions
        * Apply linear (To be upgraded?) outer boundary conditions to the vector potential and Valencia three-velocity. [Tutorial](Tutorial-GiRaFFE_NRPy-BCs.ipynb)

**A note on Validation:** All currently-written initial data modules have been validated against the original `GiRaFFE`, but this is not adequately documented.

**List of Critical Papers:**
1. The original `GiRaFFE` paper: [arxiv:1704.00599](https://arxiv.org/abs/1704.00599)
    * This project is primarily based on the older `GiRaFFE` code. This, in turn, depends on some other works:
        1. `IllinoisGRMHD`: [arxiv:1501.07276](https://arxiv.org/abs/1501.07276)
            * Large parts of `GiRaFFE` were written as simplifications of `IllinoisGRMHD`, because GRFFE is a special case of GRMHD
        1. Paschalidis, et al. (2013): [arxiv:1310.3274](https://arxiv.org/abs/1310.3274)
            * Provides formulae used for C2P and P2C solvers and the evolution equation in terms of the densitized Poynting flux. 